# 🇦🇷 Dashboard Macro Argentina

Monitoreo de indicadores económicos en tiempo real.

In [ ]:
import pandas as pd
import numpy as np
import requests
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta

# Configuración global de estilo
COLORS = {
    'primary': '#1a1a2e',
    'secondary': '#16213e', 
    'accent': '#0f3460',
    'highlight': '#e94560',
    'gold': '#f1c40f',
    'success': '#00d9a5',
    'warning': '#ff6b35',
    'blue': '#4361ee',
    'purple': '#7209b7',
    'cyan': '#4cc9f0',
    'text': '#ffffff',
    'text_muted': '#a0a0a0',
    'bg': '#0d1117',
    'card': '#161b22'
}

TEMPLATE = 'plotly_dark'

pd.options.display.float_format = '{:,.2f}'.format

## 📊 Obtención de Datos

In [ ]:
def get_dolar_hoy():
    """Cotizaciones actuales del dólar."""
    try:
        response = requests.get('https://dolarapi.com/v1/dolares', timeout=10)
        data = response.json()
        df = pd.DataFrame(data)
        df = df[['nombre', 'compra', 'venta', 'fechaActualizacion']]
        df.columns = ['Tipo', 'Compra', 'Venta', 'Actualización']
        return df
    except Exception as e:
        print(f"Error: {e}")
        return None

def get_dolar_historico(dias=365):
    """Serie histórica del dólar."""
    try:
        url = f'https://api.bluelytics.com.ar/v2/evolution.json?days={dias}'
        response = requests.get(url, timeout=10)
        data = response.json()
        
        df = pd.DataFrame(data)
        df['date'] = pd.to_datetime(df['date'])
        df = df.set_index('date').sort_index()
        
        df['blue_venta'] = df['blue'].apply(lambda x: x.get('value_sell') if isinstance(x, dict) else None)
        df['oficial_venta'] = df['oficial'].apply(lambda x: x.get('value_sell') if isinstance(x, dict) else None)
        df['blue_compra'] = df['blue'].apply(lambda x: x.get('value_buy') if isinstance(x, dict) else None)
        df['oficial_compra'] = df['oficial'].apply(lambda x: x.get('value_buy') if isinstance(x, dict) else None)
        
        return df[['blue_venta', 'oficial_venta', 'blue_compra', 'oficial_compra']].dropna()
    except Exception as e:
        print(f"Error: {e}")
        return None

# Cargar datos
print("⏳ Cargando datos...")
dolar_hoy = get_dolar_hoy()
dolar_hist = get_dolar_historico(dias=365)
print("✅ Datos cargados")

In [ ]:
# Datos de inflación 2024 (INDEC)
inflacion_data = {
    'mes': pd.date_range('2024-01-01', periods=12, freq='MS'),
    'ipc_mensual': [20.6, 13.2, 11.0, 8.8, 4.2, 4.6, 4.0, 4.2, 3.5, 2.7, 2.4, 2.7],
    'ipc_nucleo': [20.2, 12.3, 9.4, 6.3, 3.7, 3.2, 2.8, 3.1, 2.9, 2.4, 2.2, 2.5]
}
inflacion = pd.DataFrame(inflacion_data)
inflacion['ipc_acumulado'] = (1 + inflacion['ipc_mensual']/100).cumprod() * 100 - 100
inflacion['mes_nombre'] = inflacion['mes'].dt.strftime('%b')

# Datos de tasas
tasas_data = {
    'Instrumento': ['Plazo Fijo', 'BADLAR', 'Política Monetaria', 'LECAP'],
    'TNA': [37.0, 34.0, 32.0, 38.0],
    'TEA': [43.8, 39.5, 37.1, 45.2]
}
tasas = pd.DataFrame(tasas_data)

---
## 💵 Panel Principal: Tipo de Cambio

In [ ]:
# KPIs principales
if dolar_hoy is not None and dolar_hist is not None:
    blue_actual = dolar_hoy[dolar_hoy['Tipo'].str.contains('Blue', case=False)]['Venta'].values[0]
    oficial_actual = dolar_hoy[dolar_hoy['Tipo'].str.contains('Oficial', case=False)]['Venta'].values[0]
    brecha_actual = (blue_actual / oficial_actual - 1) * 100
    
    # Variación mensual
    blue_30d = dolar_hist['blue_venta'].iloc[-30] if len(dolar_hist) > 30 else dolar_hist['blue_venta'].iloc[0]
    var_blue_30d = (blue_actual / blue_30d - 1) * 100
    
    # Cards con KPIs
    fig_kpis = go.Figure()
    
    fig_kpis.add_trace(go.Indicator(
        mode="number+delta",
        value=blue_actual,
        number={'prefix': "$", 'font': {'size': 48, 'color': COLORS['gold']}},
        delta={'position': "bottom", 'reference': blue_30d, 'relative': True,
               'valueformat': '.1%', 'font': {'size': 18}},
        title={'text': "<b>DÓLAR BLUE</b><br><span style='font-size:14px;color:gray'>Venta</span>",
               'font': {'size': 20, 'color': COLORS['text']}},
        domain={'x': [0, 0.3], 'y': [0, 1]}
    ))
    
    fig_kpis.add_trace(go.Indicator(
        mode="number",
        value=oficial_actual,
        number={'prefix': "$", 'font': {'size': 48, 'color': COLORS['cyan']}},
        title={'text': "<b>DÓLAR OFICIAL</b><br><span style='font-size:14px;color:gray'>Venta</span>",
               'font': {'size': 20, 'color': COLORS['text']}},
        domain={'x': [0.35, 0.65], 'y': [0, 1]}
    ))
    
    fig_kpis.add_trace(go.Indicator(
        mode="number+delta",
        value=brecha_actual,
        number={'suffix': "%", 'font': {'size': 48, 'color': COLORS['highlight']}},
        delta={'position': "bottom", 'reference': 30, 'relative': False,
               'suffix': ' vs 30%', 'font': {'size': 14}},
        title={'text': "<b>BRECHA</b><br><span style='font-size:14px;color:gray'>Blue vs Oficial</span>",
               'font': {'size': 20, 'color': COLORS['text']}},
        domain={'x': [0.7, 1], 'y': [0, 1]}
    ))
    
    fig_kpis.update_layout(
        paper_bgcolor=COLORS['bg'],
        plot_bgcolor=COLORS['bg'],
        height=200,
        margin=dict(t=60, b=20, l=20, r=20),
        title=dict(
            text=f"<b>💵 COTIZACIONES AL {datetime.now().strftime('%d/%m/%Y %H:%M')}</b>",
            font=dict(size=16, color=COLORS['text_muted']),
            x=0.5
        )
    )
    
    fig_kpis.show()

In [ ]:
# Evolución histórica con área sombreada
if dolar_hist is not None:
    fig_evol = go.Figure()
    
    fig_evol.add_trace(go.Scatter(
        x=dolar_hist.index,
        y=dolar_hist['blue_venta'],
        name='Blue Venta',
        line=dict(color=COLORS['gold'], width=3),
        hovertemplate='Blue: $%{y:,.0f}<extra></extra>'
    ))
    
    fig_evol.add_trace(go.Scatter(
        x=dolar_hist.index,
        y=dolar_hist['blue_compra'],
        name='Blue Compra',
        line=dict(color=COLORS['gold'], width=1, dash='dot'),
        fill='tonexty',
        fillcolor='rgba(241, 196, 15, 0.1)',
        hovertemplate='Blue Compra: $%{y:,.0f}<extra></extra>'
    ))
    
    fig_evol.add_trace(go.Scatter(
        x=dolar_hist.index,
        y=dolar_hist['oficial_venta'],
        name='Oficial Venta',
        line=dict(color=COLORS['cyan'], width=3),
        hovertemplate='Oficial: $%{y:,.0f}<extra></extra>'
    ))
    
    fig_evol.update_layout(
        title=dict(
            text='<b>📈 EVOLUCIÓN DEL TIPO DE CAMBIO</b>',
            font=dict(size=20, color=COLORS['text']),
            x=0.5
        ),
        template=TEMPLATE,
        paper_bgcolor=COLORS['bg'],
        plot_bgcolor=COLORS['card'],
        height=450,
        hovermode='x unified',
        legend=dict(
            orientation='h',
            yanchor='bottom',
            y=1.02,
            xanchor='center',
            x=0.5,
            font=dict(size=12)
        ),
        xaxis=dict(
            title='',
            showgrid=True,
            gridcolor='rgba(255,255,255,0.1)',
            rangeslider=dict(visible=True, thickness=0.05)
        ),
        yaxis=dict(
            title='ARS',
            showgrid=True,
            gridcolor='rgba(255,255,255,0.1)',
            tickprefix='$'
        ),
        margin=dict(t=80, b=60)
    )
    
    fig_evol.show()

In [ ]:
# Brecha cambiaria con zonas
if dolar_hist is not None:
    dolar_hist['brecha'] = (dolar_hist['blue_venta'] / dolar_hist['oficial_venta'] - 1) * 100
    brecha_promedio = dolar_hist['brecha'].mean()
    
    fig_brecha = go.Figure()
    
    fig_brecha.add_hrect(y0=50, y1=dolar_hist['brecha'].max()+10,
                         fillcolor="rgba(233, 69, 96, 0.15)", line_width=0,
                         annotation_text="Zona alta", annotation_position="top right")
    
    fig_brecha.add_hrect(y0=20, y1=50,
                         fillcolor="rgba(255, 107, 53, 0.1)", line_width=0)
    
    fig_brecha.add_hrect(y0=0, y1=20,
                         fillcolor="rgba(0, 217, 165, 0.1)", line_width=0,
                         annotation_text="Zona normal", annotation_position="bottom right")
    
    fig_brecha.add_trace(go.Scatter(
        x=dolar_hist.index,
        y=dolar_hist['brecha'],
        mode='lines',
        name='Brecha',
        line=dict(color=COLORS['highlight'], width=2.5),
        fill='tozeroy',
        fillcolor='rgba(233, 69, 96, 0.2)',
        hovertemplate='%{x}<br>Brecha: %{y:.1f}%<extra></extra>'
    ))
    
    fig_brecha.add_hline(
        y=brecha_promedio,
        line_dash='dash',
        line_color=COLORS['text_muted'],
        annotation_text=f'Promedio: {brecha_promedio:.1f}%',
        annotation_font_color=COLORS['text_muted']
    )
    
    fig_brecha.update_layout(
        title=dict(
            text='<b>📊 BRECHA CAMBIARIA</b>',
            font=dict(size=20, color=COLORS['text']),
            x=0.5
        ),
        template=TEMPLATE,
        paper_bgcolor=COLORS['bg'],
        plot_bgcolor=COLORS['card'],
        height=400,
        showlegend=False,
        xaxis=dict(title='', gridcolor='rgba(255,255,255,0.1)'),
        yaxis=dict(title='Brecha (%)', gridcolor='rgba(255,255,255,0.1)', ticksuffix='%'),
        margin=dict(t=80, b=40)
    )
    
    fig_brecha.show()

---
## 📈 Inflación

In [ ]:
# KPIs de inflación
inflacion_actual = inflacion['ipc_mensual'].iloc[-1]
inflacion_acum = inflacion['ipc_acumulado'].iloc[-1]
inflacion_anualizada = ((1 + inflacion_actual/100)**12 - 1) * 100

fig_inf_kpi = go.Figure()

fig_inf_kpi.add_trace(go.Indicator(
    mode="number+delta",
    value=inflacion_actual,
    number={'suffix': "%", 'font': {'size': 52, 'color': COLORS['warning']}},
    delta={'position': "bottom", 'reference': inflacion['ipc_mensual'].iloc[-2],
           'valueformat': '.1f', 'suffix': ' pp', 'font': {'size': 16}},
    title={'text': "<b>INFLACIÓN MENSUAL</b><br><span style='font-size:14px;color:gray'>Último mes</span>",
           'font': {'size': 18}},
    domain={'x': [0, 0.32], 'y': [0, 1]}
))

fig_inf_kpi.add_trace(go.Indicator(
    mode="number",
    value=inflacion_acum,
    number={'suffix': "%", 'font': {'size': 52, 'color': COLORS['purple']}},
    title={'text': "<b>ACUMULADA 2024</b><br><span style='font-size:14px;color:gray'>Ene-Dic</span>",
           'font': {'size': 18}},
    domain={'x': [0.34, 0.66], 'y': [0, 1]}
))

fig_inf_kpi.add_trace(go.Indicator(
    mode="number",
    value=inflacion_anualizada,
    number={'suffix': "%", 'font': {'size': 52, 'color': COLORS['cyan']}},
    title={'text': "<b>ANUALIZADA</b><br><span style='font-size:14px;color:gray'>Proyección</span>",
           'font': {'size': 18}},
    domain={'x': [0.68, 1], 'y': [0, 1]}
))

fig_inf_kpi.update_layout(
    paper_bgcolor=COLORS['bg'],
    height=180,
    margin=dict(t=50, b=20)
)

fig_inf_kpi.show()

In [ ]:
# Gráfico combinado: barras + línea
fig_inflacion = make_subplots(
    rows=1, cols=2,
    column_widths=[0.6, 0.4],
    specs=[[{"type": "bar"}, {"type": "pie"}]],
    subplot_titles=('<b>IPC Mensual 2024</b>', '<b>Composición Semestre</b>')
)

colors_gradient = [COLORS['highlight'] if x > 10 else COLORS['warning'] if x > 5 else COLORS['success'] 
                   for x in inflacion['ipc_mensual']]

fig_inflacion.add_trace(
    go.Bar(
        x=inflacion['mes_nombre'],
        y=inflacion['ipc_mensual'],
        marker_color=colors_gradient,
        text=[f'{x:.1f}%' for x in inflacion['ipc_mensual']],
        textposition='outside',
        textfont=dict(size=11, color=COLORS['text']),
        hovertemplate='%{x}<br>IPC: %{y:.1f}%<extra></extra>',
        name='IPC Mensual'
    ),
    row=1, col=1
)

fig_inflacion.add_trace(
    go.Scatter(
        x=inflacion['mes_nombre'],
        y=inflacion['ipc_nucleo'],
        mode='lines+markers',
        name='IPC Núcleo',
        line=dict(color=COLORS['cyan'], width=3),
        marker=dict(size=8)
    ),
    row=1, col=1
)

sem2 = inflacion[inflacion['mes'] >= '2024-07-01']
fig_inflacion.add_trace(
    go.Pie(
        labels=sem2['mes_nombre'],
        values=sem2['ipc_mensual'],
        hole=0.5,
        marker_colors=[COLORS['blue'], COLORS['purple'], COLORS['cyan'], 
                      COLORS['success'], COLORS['gold'], COLORS['highlight']],
        textinfo='label+percent',
        textfont=dict(size=11)
    ),
    row=1, col=2
)

fig_inflacion.update_layout(
    title=dict(
        text='<b>📈 INFLACIÓN ARGENTINA 2024</b>',
        font=dict(size=22, color=COLORS['text']),
        x=0.5
    ),
    template=TEMPLATE,
    paper_bgcolor=COLORS['bg'],
    plot_bgcolor=COLORS['card'],
    height=450,
    showlegend=True,
    legend=dict(orientation='h', y=-0.15, x=0.5, xanchor='center'),
    margin=dict(t=100)
)

fig_inflacion.update_xaxes(gridcolor='rgba(255,255,255,0.1)', row=1, col=1)
fig_inflacion.update_yaxes(gridcolor='rgba(255,255,255,0.1)', ticksuffix='%', row=1, col=1)

fig_inflacion.show()

---
## 💰 Tasas de Interés

In [ ]:
# Tasas vs Inflación
tasa_pf = tasas[tasas['Instrumento'] == 'Plazo Fijo']['TEA'].values[0]
tasa_real = tasa_pf - inflacion_anualizada

fig_tasas = go.Figure()

instruments = tasas['Instrumento'].tolist() + ['Inflación Anualizada', 'Tasa Real PF']
values = tasas['TEA'].tolist() + [inflacion_anualizada, tasa_real]
colors_bars = [COLORS['blue'], COLORS['purple'], COLORS['cyan'], COLORS['gold'], 
               COLORS['highlight'], COLORS['success'] if tasa_real > 0 else COLORS['warning']]

fig_tasas.add_trace(go.Bar(
    y=instruments,
    x=values,
    orientation='h',
    marker=dict(
        color=colors_bars,
        line=dict(color='rgba(255,255,255,0.3)', width=1)
    ),
    text=[f'{v:.1f}%' for v in values],
    textposition='outside',
    textfont=dict(size=14, color=COLORS['text']),
    hovertemplate='%{y}: %{x:.1f}%<extra></extra>'
))

fig_tasas.add_vline(
    x=inflacion_anualizada,
    line_dash='dash',
    line_color=COLORS['highlight'],
    line_width=2
)

fig_tasas.update_layout(
    title=dict(
        text='<b>💰 TASAS DE INTERÉS VS INFLACIÓN</b>',
        font=dict(size=22, color=COLORS['text']),
        x=0.5
    ),
    template=TEMPLATE,
    paper_bgcolor=COLORS['bg'],
    plot_bgcolor=COLORS['card'],
    height=400,
    xaxis=dict(
        title='Tasa Efectiva Anual (%)',
        gridcolor='rgba(255,255,255,0.1)',
        ticksuffix='%'
    ),
    yaxis=dict(gridcolor='rgba(255,255,255,0.1)'),
    margin=dict(l=150, t=80)
)

fig_tasas.show()

---
## 🎯 Dashboard Consolidado

In [ ]:
# Dashboard final
fig_dashboard = make_subplots(
    rows=3, cols=2,
    specs=[
        [{"type": "indicator", "colspan": 2}, None],
        [{"type": "scatter"}, {"type": "bar"}],
        [{"type": "scatter"}, {"type": "bar"}]
    ],
    row_heights=[0.2, 0.4, 0.4],
    vertical_spacing=0.08,
    horizontal_spacing=0.08,
    subplot_titles=(
        '',
        '<b>Tipo de Cambio</b>', '<b>Inflación Mensual</b>',
        '<b>Brecha Cambiaria</b>', '<b>Tasas de Interés</b>'
    )
)

if dolar_hoy is not None:
    blue_val = dolar_hoy[dolar_hoy['Tipo'].str.contains('Blue', case=False)]['Venta'].values[0]
    
    fig_dashboard.add_trace(
        go.Indicator(
            mode="number",
            value=blue_val,
            number={'prefix': "DÓLAR BLUE: $", 'font': {'size': 36, 'color': COLORS['gold']}},
            domain={'x': [0, 1], 'y': [0, 1]}
        ),
        row=1, col=1
    )

if dolar_hist is not None:
    fig_dashboard.add_trace(
        go.Scatter(x=dolar_hist.index, y=dolar_hist['blue_venta'],
                   name='Blue', line=dict(color=COLORS['gold'], width=2)),
        row=2, col=1
    )
    fig_dashboard.add_trace(
        go.Scatter(x=dolar_hist.index, y=dolar_hist['oficial_venta'],
                   name='Oficial', line=dict(color=COLORS['cyan'], width=2)),
        row=2, col=1
    )

fig_dashboard.add_trace(
    go.Bar(x=inflacion['mes_nombre'], y=inflacion['ipc_mensual'],
           marker_color=COLORS['warning'], name='IPC'),
    row=2, col=2
)

if dolar_hist is not None:
    fig_dashboard.add_trace(
        go.Scatter(x=dolar_hist.index, y=dolar_hist['brecha'],
                   fill='tozeroy', fillcolor='rgba(233, 69, 96, 0.3)',
                   line=dict(color=COLORS['highlight'], width=2), name='Brecha'),
        row=3, col=1
    )

fig_dashboard.add_trace(
    go.Bar(x=tasas['Instrumento'], y=tasas['TEA'],
           marker_color=[COLORS['blue'], COLORS['purple'], COLORS['cyan'], COLORS['gold']],
           name='TEA'),
    row=3, col=2
)

fig_dashboard.update_layout(
    title=dict(
        text=f'<b>🇦🇷 DASHBOARD MACRO ARGENTINA</b><br><span style="font-size:14px;color:{COLORS["text_muted"]}">Actualizado: {datetime.now().strftime("%d/%m/%Y %H:%M")}</span>',
        font=dict(size=26, color=COLORS['text']),
        x=0.5
    ),
    template=TEMPLATE,
    paper_bgcolor=COLORS['bg'],
    plot_bgcolor=COLORS['card'],
    height=900,
    showlegend=False,
    margin=dict(t=120, b=40)
)

fig_dashboard.update_xaxes(gridcolor='rgba(255,255,255,0.05)')
fig_dashboard.update_yaxes(gridcolor='rgba(255,255,255,0.05)')

fig_dashboard.show()

In [ ]:
# Exportar a HTML
fig_dashboard.write_html('../dashboard_macro_argentina.html', include_plotlyjs='cdn')
print("✅ Dashboard exportado a 'dashboard_macro_argentina.html'")

---
## 📋 Resumen Ejecutivo

In [ ]:
print("\n" + "═"*70)
print("                    🇦🇷 RESUMEN MACRO ARGENTINA")
print("═"*70)
print(f"                    {datetime.now().strftime('%d de %B de %Y')}")
print("═"*70)

if dolar_hoy is not None:
    print("\n💵 TIPO DE CAMBIO")
    print("─"*40)
    for _, row in dolar_hoy.iterrows():
        print(f"   {row['Tipo']:.<25} ${row['Venta']:>10,.0f}")

if dolar_hist is not None:
    print(f"\n   Brecha actual: {brecha_actual:.1f}%")
    print(f"   Brecha promedio (12m): {dolar_hist['brecha'].mean():.1f}%")

print("\n📈 INFLACIÓN")
print("─"*40)
print(f"   Mensual (último mes):    {inflacion_actual:.1f}%")
print(f"   Acumulada 2024:          {inflacion_acum:.1f}%")
print(f"   Anualizada (proyección): {inflacion_anualizada:.1f}%")

print("\n💰 TASAS DE INTERÉS (TEA)")
print("─"*40)
for _, row in tasas.iterrows():
    print(f"   {row['Instrumento']:.<25} {row['TEA']:>6.1f}%")
print(f"\n   Tasa real (PF):          {tasa_real:+.1f}%")

print("\n" + "═"*70)
print("                    Leonardo Gutierrez Ferrara")
print("                    lgf-consulting.com")
print("═"*70)